In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import os
import squidpy as sq
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import catboost
import json


from tqdm import tqdm
import sys
sys.path.append('/Users/rssantanu/Desktop/codebase/constrained_FM')


import argparse

max_rows = 1000000
pd.set_option('display.max_rows', max_rows)
pd.set_option('display.max_columns', max_rows)
pd.set_option('display.width', max_rows)


In [58]:
scRNA = ad.read_h5ad(os.path.join('/Users/rssantanu/Desktop/codebase/constrained_FM', 'datasets/h5ad_processed_datasets', 'GSE232025_stereoseq.h5ad'))


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [59]:
import liana as li

In [60]:
import re

def extract_symbol(name):
    # Try to find a gene symbol in the string
    # Look for patterns like 'SYMBOL[nr]' or 'SYMBOL[hs]' or 'SYMBOL |'
    match = re.search(r'([A-Z0-9\-]+)(?:\[.*?\])?\s*\|', name)
    if match:
        return match.group(1)
    # If not found, try to find a symbol before a space or bracket
    match = re.match(r'([A-Z0-9\-]+)', name)
    if match:
        return match.group(1)
    return None

# Apply to your var_names
symbols = [extract_symbol(x) for x in scRNA.var_names]
scRNA.var['gene_symbol'] = symbols
# Filter out None or nan
scRNA = scRNA[:, [s is not None and s != 'nan' for s in symbols]]
scRNA.var_names = scRNA.var['gene_symbol']

/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [61]:
# for all the days

li.method.rank_aggregate(
    adata=scRNA,
    groupby='celltype',
    resource_name='consensus',
    use_raw=False,         # <--- add this line
    verbose=True
)

scRNA_by_day= []

for stage in scRNA.obs['day'].unique():
    adata_stage = scRNA[scRNA.obs['day'] == stage].copy()
    li.method.rank_aggregate(
        adata=adata_stage,
        groupby='celltype',
        resource_name='consensus',
        use_raw=False,
        verbose=True
    )
    # Save or analyze results for this stage
    # adata_stage.uns['liana_res'].to_csv(f'liana_results_stage_{stage}.csv')
    scRNA_by_day.append(adata_stage)



Using resource `consensus`.
Using `.X`!
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable nam

Generating ligand-receptor stats for 28757 samples and 119 features
Assuming that counts were `natural` log-normalized!


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/liana/method/sc/_liana_pipe.py:262: ImplicitModificationWarning: Setting element `.layers['scaled']` of view, initializing view as actual.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Running CellPhoneDB


100%|██████████| 1000/1000 [00:06<00:00, 144.56it/s]


...

Generating ligand-receptor stats for 8243 samples and 119 features
Assuming that counts were `natural` log-normalized!


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/liana/method/sc/_liana_pipe.py:262: ImplicitModificationWarning: Setting element `.layers['scaled']` of view, initializing view as actual.


Running CellPhoneDB


100%|██████████| 1000/1000 [00:03<00:00, 327.51it/s]


Running Connectome
Running log2FC
Running NATMI
Running SingleCellSignalR


In [62]:
liana_df= scRNA.uns['liana_res']

In [63]:
# liana_df[(liana_df['source']=='Immature dpEX') & (liana_df['target']=='Immature dpEX')]

liana_df[(liana_df['source']=='Immature nptxEX') & (liana_df['target']=='Immature mpEX')]


,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,specificity_rank,magnitude_rank
2393,Immature nptxEX,Immature mpEX,NPTX2,NPTXR,2.445291,0.000,5.117473,1.550619,1.841359,0.009017,0.934743,0.001578,0.000004
2392,Immature nptxEX,Immature mpEX,NPTX1,NPTXR,2.168506,0.000,3.249888,1.877815,1.868729,0.021776,0.919452,0.000017,0.000182
2398,Immature nptxEX,Immature mpEX,SFRP1,FZD2,0.095272,1.000,0.009066,-0.200893,-0.484970,0.000228,0.376135,1.000000,0.015158
2388,Immature nptxEX,Immature mpEX,ADCYAP1,ADCYAP1R1,1.163083,0.000,0.723062,0.706907,0.655797,0.002324,0.843365,0.070750,0.046099
2389,Immature nptxEX,Immature mpEX,KL,FGFR1,0.115016,0.851,0.013220,-0.031534,-0.061911,0.000702,0.421313,1.000000,0.212410
2390,Immature nptxEX,Immature mpEX,L1CAM,CD9,0.843674,0.000,0.318090,0.444309,0.401623,0.002309,0.781239,0.072401,0.316222
2397,Immature nptxEX,Immature mpEX,SEMA3A,NRP2,0.716193,0.000,0.257679,0.708747,0.665567,0.003956,0.762709,0.030461,0.460592
2395,Immature nptxEX,Immature mpEX,NXPH1,NRXN3,0.382300,0.000,0.130256,0.253638,0.219981,0.001562,0.695611,0.250704,0.536316
2394,Immature nptxEX,Immature mpEX,NPY,NPY2R,0.162105,0.000,0.025562,0.356672,0.228878,0.003041,0.503071,0.081092,0.917568
2396,Immature nptxEX,Immature mpEX,PTN,PTPRB,0.308588,0.016,0.061919,0.199425,-0.060762,0.003768,0.611743,0.675823,0.923635


In [64]:
# import liana as li

# li.pl.dotplot(
#     adata=scRNA,
#     colour='lr_means',           # or another score column
#     size='specificity_rank',     # or another ranking/stat column
#     inverse_size=True,           # small p-values = big dots
#     source_labels=['Immature dpEX', 'Immature mpEX'],  # replace with your cell types
#     target_labels=['Immature nptxEX', 'Immature mpEX'],  # replace with your cell types
#     figure_size=(9, 5),
#     filter_fun=lambda x: x['cellphone_pvals'] <= 0.05,  # filter by significance
#     cmap='plasma'
# )

In [65]:
# scRNA_by_day[2].uns['liana_res']

In [66]:
# for scrna_day in scRNA_by_day[2]:

#     li.pl.dotplot(
#         adata=scrna_day,
#         colour='lr_means',           # or another score column
#         size='specificity_rank',     # or another ranking/stat column
#         inverse_size=True,           # small p-values = big dots
#         source_labels=['Immature dpEX', 'Immature mpEX'],  # replace with your cell types
#         target_labels=['Immature nptxEX', 'Immature mpEX'],  # replace with your cell types
#         figure_size=(9, 5),
#         filter_fun=lambda x: x['cellphone_pvals'] <= 0.05,  # filter by significance
#         cmap='plasma'
#     )

In [67]:
def get_communication_matrix(adata, all_celltypes):

    communication_dict = {}
    liana_df = adata.uns['liana_res']
    for celltype in all_celltypes:
        communication_dict[celltype] = []
        for celltype_2 in all_celltypes:
            if celltype in liana_df['source'].unique() and celltype_2 in liana_df['target'].unique():
                communication_dict[celltype].append(liana_df[(liana_df['source'] == celltype) & (liana_df['target'] == celltype_2)]['lrscore'].values.sum())
            else:
                communication_dict[celltype].append(0)

    cc_comm_df= pd.DataFrame(communication_dict, index=all_celltypes, columns=all_celltypes)

    return cc_comm_df

In [68]:
all_celltypes = scRNA.obs['celltype'].unique()

In [69]:
all_data_comm= get_communication_matrix(scRNA, all_celltypes)

In [70]:
data_comm_0 = get_communication_matrix(scRNA_by_day[0], all_celltypes)
data_comm_1 = get_communication_matrix(scRNA_by_day[1], all_celltypes)
data_comm_2 = get_communication_matrix(scRNA_by_day[2], all_celltypes)
data_comm_3 = get_communication_matrix(scRNA_by_day[3], all_celltypes)
data_comm_4 = get_communication_matrix(scRNA_by_day[4], all_celltypes)








In [71]:
data_comm_4

,Immature nptxEX,VLMC,CP,dNBL1,Immature MSN,dNBL2,Immature CMPN,dEGC,dNBL3,Immature cckIN,Immature dpEX,dNBL4,Immature DMIN,wntEGC,sfrpEGC,Immature mpEX,dNBL5,nptxEX,MSN,sstIN,Unknown,tlNBL,dpEX,MCG,CMPN,npyIN,cckIN,scgnIN,ribEGC,mpEX,ntng1IN,mpIN
Immature nptxEX,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VLMC,0,4.111259,3.621933,0,0,0,0,0,0,0,0,0,0,9.000273,8.895769,0,0,5.554606,4.949380,4.702588,0,6.463320,4.226785,0,4.979526,0,4.906732,5.136333,4.505177,4.155336,5.964667,4.177746
CP,0,2.102047,1.611226,0,0,0,0,0,0,0,0,0,0,4.979166,4.937141,0,0,3.723115,3.110575,2.879082,0,4.518277,2.332543,0,2.602777,0,3.119812,2.703276,1.769893,2.304402,3.621109,2.343680
dNBL1,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Immature MSN,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dNBL2,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Immature CMPN,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dEGC,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dNBL3,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Immature cckIN,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
